In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
import click 
import umap
import os

2023-12-09 20:57:37.814552: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/DAVIDSON/dmkurdydyk/.conda/envs/aot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Base directory
base_dir = os.getcwd()

# Paths
data_raw_where = base_dir + '/data/' # where the raw data(before sampling) were stored
data_sampled_where = base_dir + '/data/data_sampled' # where the sampled data were stored
weights_path = base_dir + '/new_weights/' # where the weights were stored
predict_where = base_dir + '/new_one_class_SVM_predict/' # Where to save predictions
umap_where = base_dir + '/umap_plots/' # where to save umap plots

# Constants
ISOTOPE = 'O16'
folder = base_dir + '/datalessthan5/'
sample_size = 512 # sample size = num_points 
d_max = 1024 # Size of latent Space

In [7]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)
def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)
class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)
    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))
    
    def get_config(self):
        return {'l2reg': self.l2reg,
               'num_features': self.num_features,
               'eye': self.eye.numpy().tolist()}
    
def tnet(inputs, num_features):
    # Initalise bias as the indentity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)
    
    x = conv_bn(inputs, 64) #64->32
    x = conv_bn(x, 128) #128 -> 64
    x = conv_bn(x, 1024) #1024 -> 512
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 512) # 512 -> 256
    x = dense_bn(x, 256) #256 -> 128
        
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

# PointNet for multi-class classification
def create_pointnet(num_points, num_classes, input_dimension=3):
    inputs = keras.Input(shape=(num_points, input_dimension))

    # T-Net layers
    x = tnet(inputs, input_dimension)

    # PointNet layers
    x = conv_bn(x, 64)
    x = conv_bn(x, 128)
    x = conv_bn(x, 1024)  # Adjust the size according to your d_max
    global_features = layers.GlobalMaxPooling1D()(x)

    # Fully connected layers for classification
    x = dense_bn(global_features, 512)
    x = layers.Dropout(0.3)(x)
    x = dense_bn(x, 256)
    x = layers.Dropout(0.3)(x)

    # Adjust the output layer according to the number of classes
    if num_classes == 2:
        outputs = layers.Dense(1, activation="sigmoid")(x)  # Use sigmoid for binary classification
    else:
        outputs = layers.Dense(num_classes, activation="softmax")(x)  # Use softmax for multi-class

    model = keras.Model(inputs=inputs, outputs=outputs, name="PointNet")
    
    return model

# Set your parameters
num_points = 512  # Example: number of points in each sample
num_classes = 3  # Set the number of classes for your task
input_dimension = 3  # Assuming 3D input data (X, Y, Z)

# Create the model
model = create_pointnet(num_points, num_classes, input_dimension)

path = weights_path + 'weights_dresser_chair_2_512_1_20_1024.npy'

npy_weights = np.load(path, allow_pickle=True)

# Initialize a counter to keep track of the weight set
weight_counter = 0

# Manually assign weights to each layer
for layer in model.layers:
    if isinstance(layer, keras.layers.Conv1D):
        # Conv1D layers expect two sets of weights: kernel and bias
        layer.set_weights([npy_weights[weight_counter], npy_weights[weight_counter + 1]])
        weight_counter += 2
    elif isinstance(layer, keras.layers.BatchNormalization):
        # BatchNormalization layers expect four sets of weights
        layer.set_weights(npy_weights[weight_counter:weight_counter + 4])
        weight_counter += 4
    elif isinstance(layer, keras.layers.Dense):
        # Dense layers expect two sets of weights: kernel and bias
        layer.set_weights([npy_weights[weight_counter], npy_weights[weight_counter + 1]])
        weight_counter += 2

ValueError: Layer weight shape (1, 64, 128) not compatible with provided weight shape (1, 64, 64)

In [9]:
npy_weights = np.load(path, allow_pickle=True)

# Verify the number of weight sets
print(f"Number of weight sets in npy file: {len(npy_weights)}")
print(f"Number of trainable weights in the model: {len(model.trainable_weights)}")

# Manually load weights for each layer
for layer, layer_weights in zip(model.layers, npy_weights):
    # Check if the layer has trainable weights
    if layer.trainable_weights:
        # Print shapes for debugging
        print(f"Layer {layer.name}: Expected {len(layer.trainable_weights)}, Found {len(layer_weights)}")
        for lw, tw in zip(layer_weights, layer.trainable_weights):
            print(f"  - Weight shape: expected {tw.shape}, found {lw.shape}")
        
        # Set the weights
        layer.set_weights(layer_weights)

Number of weight sets in npy file: 108
Number of trainable weights in the model: 44
Layer conv1d_12: Expected 2, Found 64
  - Weight shape: expected (1, 3, 64), found ()
  - Weight shape: expected (64,), found ()


ValueError: You called `set_weights(weights)` on layer "conv1d_12" with a weight list of length 64, but the layer was expecting 2 weights. Provided weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0...

In [5]:
npy_weights = np.load(path, allow_pickle=True)

# Inspect the weights for each set in the npy file
for i, weight_set in enumerate(npy_weights):
    print(f"Weight set {i}:")
    if isinstance(weight_set, np.ndarray):
        print(f"  Shape: {weight_set.shape}")
    else:
        print("  Multiple weights:")
        for w in weight_set:
            print(f"    Shape: {w.shape}")

Weight set 0:
  Shape: (1, 3, 64)
Weight set 1:
  Shape: (64,)
Weight set 2:
  Shape: (64,)
Weight set 3:
  Shape: (64,)
Weight set 4:
  Shape: (64,)
Weight set 5:
  Shape: (64,)
Weight set 6:
  Shape: (1, 64, 128)
Weight set 7:
  Shape: (128,)
Weight set 8:
  Shape: (128,)
Weight set 9:
  Shape: (128,)
Weight set 10:
  Shape: (128,)
Weight set 11:
  Shape: (128,)
Weight set 12:
  Shape: (1, 128, 1024)
Weight set 13:
  Shape: (1024,)
Weight set 14:
  Shape: (1024,)
Weight set 15:
  Shape: (1024,)
Weight set 16:
  Shape: (1024,)
Weight set 17:
  Shape: (1024,)
Weight set 18:
  Shape: (1024, 512)
Weight set 19:
  Shape: (512,)
Weight set 20:
  Shape: (512,)
Weight set 21:
  Shape: (512,)
Weight set 22:
  Shape: (512,)
Weight set 23:
  Shape: (512,)
Weight set 24:
  Shape: (512, 256)
Weight set 25:
  Shape: (256,)
Weight set 26:
  Shape: (256,)
Weight set 27:
  Shape: (256,)
Weight set 28:
  Shape: (256,)
Weight set 29:
  Shape: (256,)
Weight set 30:
  Shape: (256, 9)
Weight set 31:
  Sha

In [ ]:
# Freeze layers if needed (optional)
# for layer in model.layers[:-N]:  # Replace N with the number of layers you want to train
#     layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()